<a href="https://colab.research.google.com/github/mar7i4ka/Neural-Networks/blob/main/Cats%26Dogs_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

path = kagglehub.dataset_download("biaiscience/dogs-vs-cats")

print("Path to dataset files:", path)

100%|██████████| 817M/817M [00:10<00:00, 85.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/biaiscience/dogs-vs-cats/versions/1


In [ ]:
import pathlib, zipfile
import torch, torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import random_split, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

dataset_dir = pathlib.Path("/root/.cache/kagglehub/datasets/biaiscience/dogs-vs-cats/versions/1")

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),                      # [0,1]
    transforms.Normalize([0.5]*3, [0.5]*3)      # [-1,1]
])

full_ds = datasets.ImageFolder(dataset_dir, transform=transform)

subset_size = 10_000
full_ds = torch.utils.data.Subset(full_ds, range(subset_size))

train_size = int(0.8*len(full_ds))
val_size   = len(full_ds) - train_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=2)
val_dl   = DataLoader(val_ds,   batch_size=256, shuffle=False, num_workers=2)


In [ ]:
class TinyFFNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*64*3, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TinyFFNN().to(device)
opt   = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

def step(dl, train=False):
    total, correct, loss_sum = 0, 0, 0.0
    model.train(train)
    for xb, yb in (tqdm(dl) if train else dl):
        xb, yb = xb.to(device), yb.float().to(device)
        logits = model(xb)
        loss = criterion(logits, yb)
        if train:
            opt.zero_grad(); loss.backward(); opt.step()
        loss_sum += loss.item()*yb.size(0)
        preds = (torch.sigmoid(logits) > 0.5)
        correct += (preds == yb.bool()).sum().item()
        total += yb.size(0)
    return loss_sum/total, correct/total

for epoch in range(10):
    train_loss, train_acc = step(train_dl, train=True)
    val_loss,   val_acc   = step(val_dl,   train=False)
    print(f"{epoch:02d}: "
          f"train loss={train_loss:.4f}, acc={train_acc:.3%} │ "
          f"val loss={val_loss:.4f}, acc={val_acc:.3%}")


100%|██████████| 63/63 [00:35<00:00,  1.80it/s]


00: train loss=0.0172, acc=98.312% │ val loss=0.0000, acc=100.000%


100%|██████████| 63/63 [00:35<00:00,  1.76it/s]


01: train loss=0.0000, acc=100.000% │ val loss=0.0000, acc=100.000%


100%|██████████| 63/63 [00:35<00:00,  1.77it/s]


...

07: train loss=0.0000, acc=100.000% │ val loss=0.0000, acc=100.000%


100%|██████████| 63/63 [00:37<00:00,  1.69it/s]


08: train loss=0.0000, acc=100.000% │ val loss=0.0000, acc=100.000%


100%|██████████| 63/63 [00:36<00:00,  1.72it/s]


09: train loss=0.0000, acc=100.000% │ val loss=0.0000, acc=100.000%
